In [ ]:
## 5.4 卷积神经网络的可视化
### 5.4.1 可视化中间激活

In [ ]:
from keras.models import load_model

model = load_model('cats_and_dogs_small_2.h5')
model.summary()

**代码清单 5-25** 预处理单张图像

In [ ]:
img_path = '/Users/niujie/.keras/datasets/cats_and_dogs_train/cats_and_dogs_small/test/cats/cat.1700.jpg'

from keras.preprocessing import image
import numpy as np

img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.  # 训练模型的输入数据都用这种方法预处理

# 形状为(1, 150, 150, 3)
print(img_tensor.shape)

**代码清单 5-26** 显示测试图像

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img_tensor[0])
plt.show()

**代码清单 5-27** 用一个输入张量和一个输出张量列表将模型实例化

In [ ]:
from keras import models

layer_outputs = [layer.output for layer in model.layers[:8]]    # 提取前8层的输出
# 创建一个模型，给定模型输入，可以返回这些输出
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

**代码清单 5-28** 以预测模式运行模型

In [ ]:
# 返回8个Numpy数组组成的列表，每个层激活对应一个Numpy数组
activations = activation_model.predict(img_tensor)

first_layer_activation = activations[0]
print(first_layer_activation.shape)

**代码清单 5-29** 将第4个通道可视化

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(first_layer_activation[0, :, :, 4], cmap='viridis')

**代码清单 5-30** 将第7个通道可视化

In [ ]:
plt.matshow(first_layer_activation[0, :, :, 7], cmap='viridis')

**代码清单 5-31** 将每个中间激活的所有通道可视化

In [ ]:
# 层的名称，可以将这些名称画到图中
layer_names = []
for layer in model.layers[:8]:
    layer_names.append(layer.name)

images_per_row = 16

# 显示特征图
for layer_name, layer_activation in zip(layer_names, activations):
    n_features = layer_activation.shape[-1]     # 特征图中的特征个数

    size = layer_activation.shape[1]            # 特征图的形状为(1, size, size, n_features)

    n_cols = n_features // images_per_row       # 在这个矩阵中将激活通道平铺
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # 将每个过滤器平铺到一个大的水平网格中
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0, :, :, col * images_per_row + row]
            # 对特征进行后处理，使其看起来更美观
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image
    
    # 显示网格
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

plt.show()

**代码清单 5-32** 为过滤器的可视化定义损失张量

In [ ]:
from keras.applications import VGG16
from keras import backend as K

model = VGG16(weights='imagenet', include_top=False)

layer_name = 'block3_conv1'
filter_index = 0

layer_output = model.get_layer(layer_name).output
loss = K.mean(layer_output[:, :, :, filter_index])

**代码清单 5-33** 获取损失相对于输入的梯度

In [ ]:
# 调用gradients返回的是一个张量列表（本例中列表长度为1）。因此，只保留第一个元素，它是一个张量
grads = K.gradients(loss, model.input)[0]

**代码清单 5-34** 梯度标准化技巧

In [ ]:
# 加上1e-5防止除0
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

**代码清单 5-35** 给定Numpy输入值，得到Numpy输出值

In [ ]:
iterate = K.function([model.input], [loss, grads])

import numpy as np
loss_value, grads_value = iterate([np.zeros((1, 150, 150, 3))])

**代码清单 5-36** 通过随机梯度下降让损失最大化

In [ ]:
# 从一张带有噪音的灰度图像开始
input_img_data = np.random.random((1, 150, 150, 3)) * 20 + 128
# 每次梯度更新的步长
step = 1.
# 运行40次梯度上升
for i in range(40):
    # 计算损失和梯度值
    loss_value, grads_value = iterate([input_img_data])
    # 沿着让损失最大化的方向调节输入图像
    input_img_data += grads_value * step

**代码清单 5-37** 将张量转换为有效图像的实用函数


In [ ]:
def deprocess_image(x):
    # 对张量做标准化，使其均值为0，标准差为0.1
    x -= x.mean()
    x / (x.std() + 1e-5)
    x *= 0.1
    # 将x裁剪(clip)到[0, 1]区间
    x += 0.5
    x = np.clip(x, 0, 1)
    # 将x转换为RGB数组
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

**代码清单 5-38** 生成过滤器可视化的函数

In [ ]:
def generate_pattern(layer_name, filter_index, size=150):
    # 构建一个损失函数，将该层第n个过滤器的激活最大化
    layer_output = model.get_layer(layer_name).output
    loss = K.mean(layer_output[:, :, :, filter_index])
    # 计算这个损失相对于输入图像的梯度
    grads = K.gradients(loss, model.input)[0]
    # 标准化技巧：将梯度标准化
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
    # 返回给定输入图像的损失和梯度
    iterate = K.function([model.input], [loss, grads])
    # 从带有噪音的灰度图像开始
    input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.

    step = 1.
    # 运行40次梯度上升
    for i in range(40):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step
    
    img = input_img_data[0]
    return deprocess_image(img)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(generate_pattern('block3_conv1', 0))
plt.show()

**代码清单 5-39** 生成某一层中所有过滤器响应模式组成的网格

In [ ]:
for layer_name in ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1']:
    size = 64
    margin = 5

    # 空图像（全黑色），用于保存结果
    results = np.zeros((8 * size + 7 * margin, 8 * size + 7 * margin, 3))

    # 遍历resutls网格的行
    for i in range(8):
        # 遍历results网格的列
        for j in range(8):
            # 生成layer_name层第i+(j*8)个过滤器的模式
            filter_img = generate_pattern(layer_name, i + (j * 8), size=size)

            # 将结果放到results网格第(i, j)个方块中
            horizontal_start = i * size + i * margin
            horizontal_end = horizontal_start + size
            vertical_start = j * size + j * margin
            vertical_end = vertical_start + size
            results[horizontal_start : horizontal_end, vertical_start : vertical_end, :] = filter_img

    # 显示results网格
    plt.figure(figsize=(20, 20))
    plt.imshow(results)
    plt.show()